In [4]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.utils import simple_preprocess
import os

In [5]:
file_path = os.path.join("data", "train.parquet")
df = pd.read_parquet(file_path)

In [6]:
nltk.download("stopwords")
nltk.download("punkt")

# Initialize stemmer and stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaleb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kaleb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
df.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]


In [8]:
df["passages"][0]

{'is_selected': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32),
 'passage_text': array(["Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.",
        "The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia 's central bank and banknote issuing authority, when the Reserve Bank Act 1959 removed the central banking functions from the Commonwealth Bank. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees 

In [15]:
# find longest length of 'passage_text' (array) in df['passages']
max_length = max(len("passage_text") for passage in df["passages"])
max_length

12

In [17]:
# get max length of 'passage_text' (array) in df['passages']
max_length = max(len(passage["passage_text"]) for passage in df["passages"])

# Extract 'passage_text' list items into separate columns
for i in range(max_length):
    df[f"passage_text_{i}"] = df["passages"].apply(
        lambda x: x["passage_text"][i] if i < len(x["passage_text"]) else None
    )

,answers,passages,query,query_id,query_type,wellFormedAnswers,passage_text_0,passage_text_1,passage_text_2,passage_text_3,passage_text_4,passage_text_5,passage_text_6,passage_text_7,passage_text_8,passage_text_9,passage_text_10,passage_text_11
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[],"Since 2007, the RBA's outstanding reputation h...",The Reserve Bank of Australia (RBA) came into ...,RBA Recognized with the 2014 Microsoft US Regi...,The inner workings of a rebuildable atomizer a...,Results-Based Accountability® (also known as R...,Results-Based Accountability® (also known as R...,"RBA uses a data-driven, decision-making proces...",vs. NetIQ Identity Manager. Risk-based authent...,"A rebuildable atomizer (RBA), often referred t...",Get To Know Us. RBA is a digital and technolog...,None,None
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[],"In his younger years, Ronald Reagan was a memb...","From Wikipedia, the free encyclopedia. A Reaga...",Ronald Reagan began his political life in the ...,Ronald Wilson Reagan (/ˈrɒnəld ˈwɪlsən ˈreɪɡən...,"When Reagan was a 'liberal Democrat'. In 1948,...","Ronald Reagan (1911-2004), a former actor and ...","1984 Re-Election. In November 1984, Ronald Rea...",None,None,None,None,None
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[],"Sydney, New South Wales, Australia is located ...",This itinerary will have you crossing the coun...,"The Sydney central business district, Sydney h...",1 Taxis to the city centre should cost approxi...,Sydney is the capital city of the Australian s...,Sydney Attractions. Sydney is home to some of ...,"Sydney lies on a submergent coastline, where t...","On your right across College Street, in the sa...",The Rocks. This is a complete listing of the s...,"Sydney is a very large city, and we haven't sp...",None,None
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[],"In regards to tile installation costs, consume...",1 Polished nickel faucets-Average cost is $400...,Enhancement and improvement costs. 1 Polished...,Granite shower tile is available at an average...,1 Higher-end tile such as granite or marble is...,"Our free calculator uses recent, trusted data ...",The cost for a typical small bathroom remodel ...,The national average for a new shower installa...,1 Install ceramic tile floor to match shower-A...,None,None,None
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[],"Conclusions: In adult body CT, dose to an orga...",The brain uses loads and loads of energy which...,Conversion disorder is a type of somatoform di...,Conversion disorder is a mental condition in w...,Conversion disorder symptoms may appear sudden...,"Glucose. Glucose is a carbohydrate, and is the...",Sometimes people with conversion disorder have...,"Glucose is a carbohydrate, and is the most imp...",None,None,None,None


In [18]:
df.head(1)

,answers,passages,query,query_id,query_type,wellFormedAnswers,passage_text_0,passage_text_1,passage_text_2,passage_text_3,passage_text_4,passage_text_5,passage_text_6,passage_text_7,passage_text_8,passage_text_9,passage_text_10,passage_text_11
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[],"Since 2007, the RBA's outstanding reputation h...",The Reserve Bank of Australia (RBA) came into ...,RBA Recognized with the 2014 Microsoft US Regi...,The inner workings of a rebuildable atomizer a...,Results-Based Accountability® (also known as R...,Results-Based Accountability® (also known as R...,"RBA uses a data-driven, decision-making proces...",vs. NetIQ Identity Manager. Risk-based authent...,"A rebuildable atomizer (RBA), often referred t...",Get To Know Us. RBA is a digital and technolog...,None,None


In [37]:
def preprocess_query(query: str) -> list[str]:
    if query is None:
        return []  # Or handle it in a different way if empty lists are not suitable

    query = query.lower()

    # Remove punctuation
    query = re.sub(f"[{string.punctuation}]", "", query)

    # Tokenize query using simple_preprocess from gensim
    tokens = simple_preprocess(
        query, deacc=True
    )  # deacc=True removes accents and punctuations

    # Remove stopwords and apply stemming
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]

    # Add start and end icons
    tokens = ["[S]"] + tokens + ["[E]"]

    return tokens

In [38]:
dft = df.head()

In [39]:
# Apply the preprocessing function to each column of interest
for i in range(max_length):
    column_name = f"passage_text_{i}"
    dft[column_name] = dft[column_name].apply(preprocess_query)

C:\Users\kaleb\AppData\Local\Temp\ipykernel_20548\2038460186.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft[column_name] = dft[column_name].apply(preprocess_query)


AttributeError: 'NoneType' object has no attribute 'lower'

In [35]:
# print(df[0])
preprocessed_queries[0]

['[S]',
 'sinc',
 'rba',
 'outstand',
 'reput',
 'affect',
 'secur',
 'npa',
 'scandal',
 'rba',
 'subsidiari',
 'involv',
 'bribe',
 'oversea',
 'offici',
 'australia',
 'might',
 'win',
 'lucr',
 'noteprint',
 'contract',
 'asset',
 'bank',
 'includ',
 'gold',
 'foreign',
 'exchang',
 'reserv',
 'australia',
 'estim',
 'net',
 'worth',
 'billion',
 'nearli',
 'rba',
 'employe',
 'work',
 'headquart',
 'sydney',
 'new',
 'south',
 'wale',
 'busi',
 'resumpt',
 'site',
 '[E]']